In [ ]:
import pandas as pd
pheno_data = pd.read_csv("/mnt/mfs/hgrcgrid/homes/xx2404/xQTL_output2/data_preprocessing/Mic/phenotype_data/rosmap_pheno.bed", sep = '\t')
pheno_list = list(pheno_data)
geno_data = pd.read_csv("/mnt/vast/hpc/csg/molecular_phenotype_calling/genotype/ROSMAP_joint_call_sample_list", sep = ' ', header = None)
geno_list = geno_data[1].values.tolist()
sample_overlap_list = [value for value in pheno_list if value in geno_list]
sample_participant_lookup  = pd.DataFrame(list(zip(sample_overlap_list, sample_overlap_list)),
               columns =['participant_id', 'sample_id'])
sample_participant_lookup

In [ ]:
ls /mnt/mfs/hgrcgrid/homes/xx2404/xQTL_output2/data_preprocessing/Mic/phenotype_data

In [ ]:

mkdir /mnt/mfs/hgrcgrid/homes/xx2404/xQTL_output2/data_preprocessing/Mic/data_production/geneomics/rosmap/geno

In [ ]:
sample_participant_lookup.to_csv("/mnt/mfs/hgrcgrid/homes/xx2404/xQTL_output2/data_preprocessing/Mic/data_production/geneomics/rosmap/geno/sample_participant_lookup.tsv", index=None, sep = '\t')
sample_participant_lookup.to_csv

In [ ]:

mkdir /mnt/mfs/hgrcgrid/homes/xx2404/xQTL_output2/data_preprocessing/Mic/data_production/geneomics/rosmap/pca

In [ ]:
sos run /mnt/mfs/hgrcgrid/homes/xx2404/xqtl-pipeline/pipeline/sample_matcher.ipynb filtered_sample_list \
        --cwd /mnt/mfs/hgrcgrid/homes/xx2404/xQTL_output2/data_preprocessing/Mic/data_production/geneomics/rosmap/geno \
        --phenoFile /mnt/mfs/hgrcgrid/homes/xx2404/xQTL_output2/data_preprocessing/Mic/phenotype_data/rosmap_pheno.bed  \
        --genoFile /mnt/vast/hpc/csg/molecular_phenotype_calling/genotype_arch/ROSMAP_NIA_WGS.leftnorm.filtered.fam  \
        --sample-participant-lookup /mnt/mfs/hgrcgrid/homes/zq2209/work/data_production/proteomics/rosmap/geno/sample_participant_lookup.tsv \
        --container /mnt/vast/hpc/csg/containers/bioinfo.sif \
        --translated_phenoFile True \
        --mem 40G -c /mnt/vast/hpc/csg/molecular_phenotype_calling/csg.yml -q csg

In [ ]:
sos run /mnt/mfs/hgrcgrid/homes/xx2404/xqtl-pipeline/pipeline/GWAS_QC.ipynb king \
    --cwd /mnt/mfs/hgrcgrid/homes/xx2404/xQTL_output2/data_preprocessing/Mic/data_production/geneomics/rosmap/geno  \
    --genoFile /mnt/vast/hpc/csg/molecular_phenotype_calling/genotype_arch/ROSMAP_NIA_WGS.leftnorm.filtered.bed \
    --name pQTL \
    --keep-samples /mnt/mfs/hgrcgrid/homes/zq2209/workdata_production/proteomics/rosmap/geno/sample_participant_lookup.filtered_geno.txt \
    --container /mnt/vast/hpc/csg/containers/bioinfo.sif --no-maximize-unrelated \
    --mem 40G -c /mnt/vast/hpc/csg/molecular_phenotype_calling/csg.yml -q csg

In [ ]:
sos run /mnt/mfs/hgrcgrid/homes/xx2404/xqtl-pipeline/pipeline/GWAS_QC.ipynb qc \
   --cwd /mnt/mfs/hgrcgrid/homes/xx2404/xQTL_output2/data_preprocessing/Mic/data_production/geneomics/rosmap/geno \
   --genoFile /mnt/vast/hpc/csg/molecular_phenotype_calling/genotype_arch/ROSMAP_NIA_WGS.leftnorm.filtered.bed \
   --mac-filter 5 \
   --container /mnt/vast/hpc/csg/containers/bioinfo.sif \
   --mem 40G -c /mnt/vast/hpc/csg/molecular_phenotype_calling/csg.yml -q csg

In [ ]:
sos run /mnt/mfs/hgrcgrid/homes/xx2404/xqtl-pipeline/pipeline/PCA.ipynb flashpca \
   --cwd /mnt/mfs/hgrcgrid/homes/xx2404/xQTL_output2/data_preprocessing/Mic/data_production/geneomics/rosmap/pca \
   --genoFile /mnt/mfs/hgrcgrid/homes/zq2209/work/data_production/proteomics/rosmap/geno/ROSMAP_NIA_WGS.leftnorm.filtered.filtered.prune.bed \
   --container /mnt/vast/hpc/csg/containers/flashpcaR.sif